Install packages and download data

In [1]:
! pip install datasets
! pip install accelerate
! mkdir data
! wget -O data/GB-GOV-1.csv https://raw.githubusercontent.com/akmiller01/practical-ml-seminar/main/data/GB-GOV-1.csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.5 MB/s eta 0:00:00
--2024-02-20 13:08:07--  https://raw.githubusercontent.com/akmiller01/practical-ml-seminar/main/data/GB-GOV-1.csv
Resolving raw.githubusercontent.com (raw.gi

Load packages

In [2]:
from transformers import pipeline
from datasets import Dataset
import pandas as pd

Load dataset, define classes, and take validation subset

In [3]:
df = pd.read_csv('data/GB-GOV-1.csv')
classes_verbalized = df.label.unique()
id2label = {i: label for i, label in enumerate(classes_verbalized)}
label2id = {id2label[i]: i for i in id2label.keys()}
dataset = Dataset.from_pandas(df).class_encode_column("label").train_test_split(
    test_size=0.3,
    stratify_by_column="label",
    shuffle=True,
)

Casting to class labels:   0%|          | 0/3878 [00:00<?, ? examples/s]

Load model as zero-shot pipeline

In [4]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33", device=0)
hypothesis_template = "This example is {}"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Iterate through validation set and score

In [5]:
correct = 0
total = 0
for i, text in enumerate(dataset['test']['text']):
    total += 1
    output = zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)
    guess = output['labels'][0]
    correct_answer = id2label[dataset['test']['label'][i]]
    if guess == correct_answer:
        correct += 1
    if i % 10 == 0:
        print("{}.{}\nAnswer: {}\nGuess: {}\n\n".format(i, text, correct_answer, guess))
    if i == 100:
        break

0.UN Convention Against Corruption :  Pilot on Implementation
Answer: Not related to climate
Guess: Not related to climate




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


10.Provision of Technical Assistance for Market Scoping and Pipeline Building in the Agricultural Sector in Zimbabwe.
Answer: Not related to climate
Guess: Not related to climate


20.Africa Progress Panel
Answer: Not related to climate
Guess: Not related to climate


30.Support to the West and Central African Council for Agricultural Research and Development (WECARD/CORAF)
Answer: Related to climate
Guess: Not related to climate


40.NONBUDSUPPFINAID for Health Policy and Support Project
Answer: Not related to climate
Guess: Not related to climate


50.Climate Smart Development for Nepal
Answer: Related to climate
Guess: Related to climate


60.UK-INDIA Partnership on National Infrastructure Investment Fund (NIIF) -  Green Growth Energy Fund
Answer: Related to climate
Guess: Not related to climate


70.Training on statistical analysis of CO2 emissions
Answer: Related to climate
Guess: Related to climate


80.Egypt CSSF Programme
Answer: Not related to climate
Guess: Not related to cli

Print accuracy

In [6]:
accuracy = correct / total
print(
    "{}%".format(
        round(accuracy * 100, 2)
    )
)

71.29%
